In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stacksample/Answers.csv
/kaggle/input/stacksample/Questions.csv
/kaggle/input/stacksample/Tags.csv
/kaggle/input/glove6b100dtxt/glove.6B.100d.txt
/kaggle/input/glove6b300dtxt/glove.6B.300d.txt
/kaggle/input/nlpword2vecembeddingspretrained/glove.6B.200d.txt
/kaggle/input/nlpword2vecembeddingspretrained/glove.6B.50d.txt
/kaggle/input/nlpword2vecembeddingspretrained/glove.6B.300d.txt
/kaggle/input/nlpword2vecembeddingspretrained/GoogleNews-vectors-negative300.bin
/kaggle/input/nlpword2vecembeddingspretrained/glove.6B.100d.txt


Importing Libraries  

In [11]:
import pandas as pd
import re
import numpy as np
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from tqdm import tqdm 
from collections import Counter  

In [2]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns

import warnings

import pickle
import time

import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss
from sklearn.cluster import KMeans

import logging

from scipy.sparse import hstack

warnings.filterwarnings("ignore")
plt.style.use('bmh') 
%matplotlib inline 

In [3]:
ques = pd.read_csv("../input/stacksample/Questions.csv",encoding="ISO-8859-1")
ques.head(3)     

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...


In [4]:
tags = pd.read_csv("../input/stacksample/Tags.csv",encoding='ISO-8859-1')
tags.head()     

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [5]:
tags['Tag'].value_counts()  

javascript           124155
java                 115212
c#                   101186
php                   98808
android               90659
                      ...  
testcasesource            1
google-floodlight         1
iecapt                    1
netfs                     1
docker-windows            1
Name: Tag, Length: 37034, dtype: int64

Grouping all tags for each question 

In [6]:
tags['Tag'] = tags['Tag'].astype(str)
grouped_tags = tags.groupby("Id")['Tag'].apply(lambda tags: ' '.join(tags))
grouped_tags.reset_index()     
print(grouped_tags.shape)    
grouped_tags.head()       

(1264216,)


Id
80                            flex actionscript-3 air
90       svn tortoisesvn branch branching-and-merging
120                               sql asp.net sitemap
180    algorithm language-agnostic colors color-space
260           c# .net scripting compiler-construction
Name: Tag, dtype: object

Removing Questions having score < 5

In [7]:
## **Run this only once**  

ques.drop(columns=['OwnerUserId', 'CreationDate', 'ClosedDate'], inplace=True)
ques = ques.merge(grouped_tags, on='Id')  

## Removing all samples having score < 5  
ques = ques[ques['Score']>5]               ## Faltu ke Questions kafi hai 

## No need of Id and score
ques.drop(columns = ['Id','Score'],inplace=True)
print(ques.shape) 
ques.head(3)                                                  

(72950, 3)


,Title,Body,Tag
0,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex actionscript-3 air
1,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn tortoisesvn branch branching-and-merging
2,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql asp.net sitemap


In [8]:
del tags
del grouped_tags    

In [9]:
ques['Tag'] = ques['Tag'].apply(lambda x: x.split())
all_tags = [item for sublist in ques['Tag'].values for item in sublist]
unique_tags = list(set(all_tags))  
len(all_tags) , len(unique_tags)                     

(224129, 14883)

Taking only 100 most common/frequent tags 

In [12]:
counter = Counter(all_tags)
most_occur = counter.most_common(100)   # Take only 100 most common tags 
len(most_occur)            

100

In [13]:
comm_tags = [i[0] for i in most_occur]  
count = [i[1] for i in most_occur]
comm_tags[:5],count[:5]   ,  len(comm_tags)   

(['c#', 'java', 'javascript', 'android', 'python'],
 [6907, 6862, 5560, 5052, 4442],
 100)

In [14]:
## Goes into array of tags for some sample question, and retains tags which are common 
def most_common(x):
    tags_filtered = []
    for i in range(0, len(x)):
        if x[i] in comm_tags:
            tags_filtered.append(x[i])
    return tags_filtered          

In [15]:
ques['Tag'] = ques['Tag'].apply(lambda x: most_common(x))

ques['Tag'] = ques['Tag'].apply(lambda x: x if len(x)>0 else None)        

In [16]:
print(ques.shape)   

## Removing samples having no common tags 
ques.dropna(subset=['Tag'], inplace=True)
print(ques.shape)    

(72950, 3)
(63167, 3)


In [26]:
print(ques.shape) 
ques.head() 

(63167, 3)


,Title,Body,Tag
1,good branch merge tutorials tortoisesvn,really good tutorials explain branch merge apa...,[svn]
2,asp.net site map,anyone get experience create sql-based asp.net...,"[sql, asp.net]"
3,function create color wheel,something pseudo-solved many time never quite ...,[algorithm]
4,add script functionality .net applications,little game write c use database back-end trad...,"[c#, .net]"
5,use nest class case,work collection class use video playback recor...,"[c++, oop, class]"


In [18]:
y_final = ques['Tag'].tolist()            
y_final[:5]      

[['svn'],
 ['sql', 'asp.net'],
 ['algorithm'],
 ['c#', '.net'],
 ['c++', 'oop', 'class']]

In [19]:
ques['Body'][1]         ## there is an HTML link  

'<p>Are there any really good tutorials explaining <a href="http://svnbook.red-bean.com/en/1.8/svn.branchmerge.html" rel="nofollow">branching and merging</a> with Apache Subversion? </p>\n\n<p>All the better if it\'s specific to TortoiseSVN client.</p>\n'

In [20]:
# Converting html to text in the body
ques['Body'] = ques['Body'].apply(lambda x: BeautifulSoup(x).get_text())     

In [21]:
ques['Body'][1]        

"Are there any really good tutorials explaining branching and merging with Apache Subversion? \nAll the better if it's specific to TortoiseSVN client.\n"

In [22]:
nltk.download('popular') 

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk

True

Following function to clean the text, remove punctutation, remove stopwords and perform lemmatization 

In [23]:
token=ToktokTokenizer()
punct = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

## Removes shortcut words like i've with i have etc.  
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

def strip_list_noempty(mylist):
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']


def clean_punct(text): 
    words=token.tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    remove_punctuation = str.maketrans(' ', ' ', punct)
    for w in words:
        if w in comm_tags: 
            punctuation_filtered.append(w)
        else:
            punctuation_filtered.append(regex.sub('', w))

    filtered_list = strip_list_noempty(punctuation_filtered)

    return ' '.join(map(str, filtered_list))

lemma=WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))

def stopWordsRemove(text):
    
    stop_words = set(stopwords.words("english"))
    
    words=token.tokenize(text)
    
    filtered = [w for w in words if not w in stop_words]
    
    return ' '.join(map(str, filtered))

In [24]:
def pre_process(new_df):
    ## Applying on Body Text 
    new_df['Body'] = new_df['Body'].apply(lambda x: clean_text(x))
    new_df['Body'] = new_df['Body'].apply(lambda x: clean_punct(x))
    new_df['Body'] = new_df['Body'].apply(lambda x: lemitizeWords(x)) 
    new_df['Body'] = new_df['Body'].apply(lambda x: stopWordsRemove(x))
    ## Applying on Ttile Text 
    new_df['Title'] = new_df['Title'].apply(lambda x: str(x))
    new_df['Title'] = new_df['Title'].apply(lambda x: clean_text(x)) 
    new_df['Title'] = new_df['Title'].apply(lambda x: clean_punct(x)) 
    new_df['Title'] = new_df['Title'].apply(lambda x: lemitizeWords(x)) 
    new_df['Title'] = new_df['Title'].apply(lambda x: stopWordsRemove(x))
    
    return new_df  

Complete pre-processed data 

In [25]:
data = pre_process(ques)
print(data.shape)    
data.head(3)                 

(63167, 3)


,Title,Body,Tag
1,good branch merge tutorials tortoisesvn,really good tutorials explain branch merge apa...,[svn]
2,asp.net site map,anyone get experience create sql-based asp.net...,"[sql, asp.net]"
3,function create color wheel,something pseudo-solved many time never quite ...,[algorithm]


In [27]:
len(data['Title'].tolist()  )       

63167

In [28]:
Title_list = []  
for sen in data['Title'].tolist():
    s = sen.split(" ")    
    Title_list.append(s)        
 
Body_list = []    
for sen in data['Body'].tolist():
    s = sen.split(" ")    
    Body_list.append(s)     
    
Body_list[0]   , Title_list[:3]          

(['really',
  'good',
  'tutorials',
  'explain',
  'branch',
  'merge',
  'apache',
  'subversion',
  'better',
  'specific',
  'tortoisesvn',
  'client'],
 [['good', 'branch', 'merge', 'tutorials', 'tortoisesvn'],
  ['asp.net', 'site', 'map'],
  ['function', 'create', 'color', 'wheel']])

# Using tf-idf features 

In [29]:
X1 = data['Title']
X2 = data['Body']     
X1.shape 

(63167,)

This is a Multi-label classification problem, so tranforming y into binary multi-label target labelling   

In [30]:
multilabel_binarizer = MultiLabelBinarizer()
y_bin = multilabel_binarizer.fit_transform(y_final)              
y_bin.shape   ,     multilabel_binarizer.classes_                               

((63167, 100),
 array(['.net', 'ajax', 'algorithm', 'android', 'angularjs', 'api',
        'arrays', 'asp.net', 'asp.net-mvc', 'asp.net-mvc-3', 'bash', 'c',
        'c#', 'c++', 'c++11', 'class', 'cocoa', 'cocoa-touch', 'css',
        'css3', 'database', 'datetime', 'debugging', 'delphi',
        'design-patterns', 'django', 'eclipse', 'emacs',
        'entity-framework', 'exception', 'facebook', 'function', 'gcc',
        'generics', 'git', 'github', 'google-chrome', 'haskell',
        'hibernate', 'html', 'html5', 'http', 'image', 'ios', 'ipad',
        'iphone', 'java', 'javascript', 'jquery', 'json', 'linq', 'linux',
        'list', 'math', 'maven', 'mongodb', 'multithreading', 'mysql',
        'node.js', 'numpy', 'objective-c', 'oop', 'optimization', 'oracle',
        'osx', 'performance', 'perl', 'php', 'postgresql', 'python', 'qt',
        'r', 'regex', 'rest', 'ruby', 'ruby-on-rails', 'ruby-on-rails-3',
        'scala', 'security', 'shell', 'spring', 'sql', 'sql-server',
      

In [31]:
y_bin  

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Peforming TF-IDF vectorization on Title and Body individually, with 1000 max features (1000 features/words with highest term frequencies) 

In [32]:
vectorizer_X1 = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       preprocessor=None,
                                       token_pattern=r"(?u)\S\S+",
                                       max_features=1000)

vectorizer_X2 = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       preprocessor=None,
                                       token_pattern=r"(?u)\S\S+",
                                       max_features=1000)          

In [33]:
X1_tfidf = vectorizer_X1.fit_transform(X1.values.astype('U')) 
X2_tfidf = vectorizer_X2.fit_transform(X2.values.astype('U'))       
X_feat_tfidf = hstack([X1_tfidf,X2_tfidf])   ## Concanating feature
print(X_feat_tfidf.shape)                               

(63167, 2000)


In [34]:
## Splitting into training and test data 
X_train, X_test, y_train, y_test = train_test_split(X_feat_tfidf, y_bin, test_size = 0.2, random_state = 0)   
X_train.shape , X_test.shape   , y_train.shape     

((50533, 2000), (12634, 2000), (50533, 100))

In [35]:
## Computes Jacard score 
def avg_jacard(y_true,y_pred):
    '''
    see https://en.wikipedia.org/wiki/Multi-label_classification#Statistics_and_evaluation_metrics
    '''
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)
    
    return jacard.mean()*100

## Prints Jacard score and hamming loss 
def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Jacard score: {}".format(avg_jacard(y_test, y_pred)))
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_test)*100))
    print("---")       

In [36]:
print("Results when using tf-idf features : ")      
print('------------------------------')  

dummy = DummyClassifier()
sgd = SGDClassifier()
lr = LogisticRegression()
svc = LinearSVC()    

for classifier in [dummy, sgd, lr, svc]: 
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test) 
    print_score(y_pred, classifier)  

Results when using tf-idf features : 
------------------------------
Clf:  DummyClassifier
Jacard score: 0.0
Hamming loss: 1.5717112553427262
---
Clf:  SGDClassifier
Jacard score: 47.5564614004538
Hamming loss: 0.9579705556435016
---
Clf:  LogisticRegression
Jacard score: 48.314468893462085
Hamming loss: 0.9712680069653316
---
Clf:  LinearSVC
Jacard score: 52.356208115666725
Hamming loss: 0.9552794047807505
---


# Using tf-idf weighted Glove Embeddings (300 dimensional)  

In [37]:
X1_arr = X1_tfidf.toarray()  
X2_arr = X2_tfidf.toarray()  
X1_arr.shape  , X2_arr.shape     

((63167, 1000), (63167, 1000))

In [38]:
print("1000 most frequent features/words/highest term frequencies")     
print('--------------------')          

print("For Title")
feat_title = vectorizer_X1.get_feature_names_out()   
print(feat_title[100:110] )      
print('------------------------------')
print("For Body") 
feat_Body = vectorizer_X2.get_feature_names_out() 
print(feat_Body[100:110] )    

1000 most frequent features/words/highest term frequencies
--------------------
For Title
['boost' 'bootstrap' 'border' 'bottom' 'box' 'branch' 'break' 'browser'
 'browsers' 'buffer']
------------------------------
For Body
['assembly' 'assign' 'assume' 'attach' 'attempt' 'attr' 'attribute'
 'authentication' 'auto' 'automatically']


In [39]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec[curr_word] = np.array(w_line[1:], dtype=np.float64)

  return word_to_vec    

word_to_vec = read_glove_vector('../input/glove6b300dtxt/glove.6B.300d.txt')                     

Here we want to take Glove embeddings for 1000 top features/words only, for each Title text and Body text  

In [40]:
emb_matrix_title = np.zeros((1000 , 300))       

for idx,word in enumerate(feat_title):     
    embedding_vector = word_to_vec.get(word)      
    if embedding_vector is not None:
        emb_matrix_title[idx, :] = embedding_vector 
        
emb_matrix_Body = np.zeros((1000 , 300))      

for idx,word in enumerate(feat_Body):        
    embedding_vector = word_to_vec.get(word)      
    if embedding_vector is not None:
        emb_matrix_Body[idx, :] = embedding_vector 
        
emb_matrix_title.shape  

(1000, 300)

Now to get td-idf weighted Glove embeddings, we multiply tf-idf matrix with embedding matrix. So each sample now will have 300 dimensional feature (Title and Body each) 

In [41]:
X1_feat = np.dot(X1_arr ,  emb_matrix_title)  
X2_feat = np.dot(X2_arr ,  emb_matrix_Body)      
print(X1_feat.shape , X2_feat.shape)        

(63167, 300) (63167, 300)


In [42]:
# X_feat = hstack([X1_feat,X2_feat])     
X_feat_300d = np.c_[X1_feat , X2_feat] 
print(X_feat_300d.shape)          

(63167, 600)


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X_feat_300d, y_bin, test_size = 0.2, random_state = 0)   
X_train.shape , X_test.shape   , y_train.shape     

((50533, 600), (12634, 600), (50533, 100))

In [44]:
print("Results for tf-idf weighted glove embeddings 300-dim")  
print('------------------------------')   
dummy = DummyClassifier()    
sgd = SGDClassifier()
lr = LogisticRegression()

for classifier in [dummy, sgd, lr]:       
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test) 
    print_score(y_pred, classifier)                            

Results for tf-idf weighted glove embeddings 300-dim
------------------------------
Clf:  DummyClassifier
Jacard score: 0.0
Hamming loss: 1.5717112553427262
---
Clf:  SGDClassifier
Jacard score: 41.912997429460944
Hamming loss: 1.1729460186797531
---
Clf:  LogisticRegression
Jacard score: 44.23157089335655
Hamming loss: 1.0845337976887763
---


SVM takes a lot of time when using this dense high dim features. That's why avoiding this. But there is scope of trying some svm packages which supports GPU implmentation, which sklern svm doesn't            

# Using tf-idf weighted Glove Embeddings (100 dimensional) 

In [45]:
word_to_vec_100 = read_glove_vector('../input/glove6b100dtxt/glove.6B.100d.txt')         

In [46]:
emb_matrix_title = np.zeros((1000 , 100))         

for idx,word in enumerate(feat_title):     
    embedding_vector = word_to_vec_100.get(word)      
    if embedding_vector is not None:
        emb_matrix_title[idx, :] = embedding_vector 
        
emb_matrix_Body = np.zeros((1000 , 100))        

for idx,word in enumerate(feat_Body):        
    embedding_vector = word_to_vec_100.get(word)        
    if embedding_vector is not None:
        emb_matrix_Body[idx, :] = embedding_vector 
        
emb_matrix_title.shape      

(1000, 100)

In [47]:
X1_feat = np.dot(X1_arr ,  emb_matrix_title)  
X2_feat = np.dot(X2_arr ,  emb_matrix_Body)      
# X_feat = hstack([X1_feat,X2_feat])     
X_feat_100d = np.c_[X1_feat , X2_feat] 
print(X_feat_100d.shape)                 

(63167, 200)


In [48]:
X_train, X_test, y_train, y_test = train_test_split(X_feat_100d, y_bin, test_size = 0.2, random_state = 0)   
X_train.shape , X_test.shape   , y_train.shape     

((50533, 200), (12634, 200), (50533, 100))

In [49]:
print("Results for tf-idf weighted glove features 100dim") 
print('------------------------------')  

dummy = DummyClassifier()
sgd = SGDClassifier()
lr = LogisticRegression()
# svc = LinearSVC()


for classifier in [dummy, sgd, lr]:   
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test) 
    print_score(y_pred, classifier)                              

Results for tf-idf weighted glove features 100dim
------------------------------
Clf:  DummyClassifier
Jacard score: 0.0
Hamming loss: 1.5717112553427262
---
Clf:  SGDClassifier
Jacard score: 30.46339431767641
Hamming loss: 1.2852619914516383
---
Clf:  LogisticRegression
Jacard score: 33.29560445359084
Hamming loss: 1.212838372645243
---


# Using pre-trained Word2Vec Embeddings  

In [50]:
# Loading the googlenews pretrained embedding
pretrained_w2v = '../input/nlpword2vecembeddingspretrained/glove.6B.100d.txt'  

def read_glove_vector(word_vec):  
  with open(word_vec, 'r', encoding='UTF-8') as f:
    words = set() 
    word_to_vec = {}   
    for line in f:
      w_line = line.split() 
      curr_word = w_line[0]
      word_to_vec[curr_word] = np.array(w_line[1:], dtype=np.float64)

  return word_to_vec    

word_to_vec = read_glove_vector(pretrained_w2v)     
len(word_to_vec) 

400000

In [51]:
w2v_matrix_title = np.zeros((1000 , 100 ))      

for idx,word in enumerate(feat_title):     
    embedding_vector = word_to_vec.get(word)      
    if embedding_vector is not None:
        w2v_matrix_title[idx, :] = embedding_vector 
        
w2v_matrix_Body = np.zeros((1000 , 100 ))      

for idx,word in enumerate(feat_Body):        
    embedding_vector = word_to_vec.get(word)      
    if embedding_vector is not None:
        w2v_matrix_Body[idx, :] = embedding_vector 
        
w2v_matrix_Body.shape       

(1000, 100)

In [52]:
X1_feat = np.dot(X1_arr ,  w2v_matrix_title)  
X2_feat = np.dot(X2_arr ,  w2v_matrix_Body )       
# X_feat = hstack([X1_feat,X2_feat])     
X_feat_w2v_pre = np.c_[X1_feat , X2_feat] 
print(X_feat_w2v_pre.shape)                     

(63167, 200)


In [53]:
X_train, X_test, y_train, y_test = train_test_split(X_feat_w2v_pre, y_bin, test_size = 0.2, random_state = 0)   
X_train.shape , X_test.shape   , y_train.shape       

((50533, 200), (12634, 200), (50533, 100))

In [54]:
print("Results for tf-idf weighted pre-trained w2v features (100-dim)")  
print('------------------------------')  

dummy = DummyClassifier()
sgd = SGDClassifier()
lr = LogisticRegression()
# svc = LinearSVC()

for classifier in [dummy, sgd, lr]:   
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)  
    print_score(y_pred, classifier)                                

Results for tf-idf weighted pre-trained w2v features (100-dim)
------------------------------
Clf:  DummyClassifier
Jacard score: 0.0
Hamming loss: 1.5717112553427262
---
Clf:  SGDClassifier
Jacard score: 30.8138846800395
Hamming loss: 1.3040208959949344
---
Clf:  LogisticRegression
Jacard score: 33.29560445359084
Hamming loss: 1.212838372645243
---


In [ ]:
# svc = LinearSVC()
# clf = OneVsRestClassifier(svc)   
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test) 
# print_score(y_pred, classifier)    

# Training Word2Vec model on text data using Genism

In [ ]:
# conda install -c conda-forge gensim    

In [62]:
from gensim.models import Word2Vec     

Input to the Word2vec has to be list of list like Title_list, where each inner list has a words from sentence tokenized 

In [63]:
model_title = Word2Vec(Title_list, min_count=3)        
model_Body = Word2Vec(Body_list, min_count=3)          

In [69]:
model_Body.wv['really']                 

array([-1.564124  , -1.8412476 ,  0.28879583, -0.42049378, -1.0304143 ,
        1.9685531 ,  1.3035272 , -0.7252116 ,  0.65933794,  0.4250319 ,
       -0.48904306, -1.0394472 , -0.11313245, -0.49694085, -1.4214902 ,
       -1.8187934 , -1.1629874 , -0.06190593, -0.9571594 ,  2.4787788 ,
       -0.02290341, -1.0628382 ,  2.2687533 ,  1.103106  ,  1.4654084 ,
       -0.9787651 , -0.19795546,  1.9252689 , -1.8423668 , -0.22675537,
       -0.16854104,  1.3008771 ,  2.0913587 , -1.7123729 , -0.69101024,
       -1.5152414 ,  0.7220079 ,  2.2958076 , -1.2556343 ,  0.59046197,
       -2.0577283 ,  0.16380861, -1.1834644 ,  0.4250607 ,  0.7335484 ,
       -1.0945364 ,  2.732179  ,  0.6100831 , -0.03248497,  0.27798465,
       -1.3344595 , -2.4514108 ,  1.6699032 , -0.25925848, -0.566183  ,
       -0.9099404 , -2.7299652 , -0.18287586, -2.4201515 ,  0.28114894,
        0.8802217 ,  1.1511499 ,  0.35467368,  1.2753938 , -0.55712926,
        1.0858672 ,  0.61823565, -1.4177705 , -1.5952297 , -0.34

In [70]:
w2v_matrix_title = np.zeros((1000 , 100 ))      

for idx,word in enumerate(feat_title):     
    embedding_vector = model_title.wv[word]          
    if embedding_vector is not None: 
        w2v_matrix_title[idx, :] = embedding_vector 
        
w2v_matrix_Body = np.zeros((1000 , 100 ))      

for idx,word in enumerate(feat_Body):        
    embedding_vector = model_Body.wv[word]        
    if embedding_vector is not None:
        w2v_matrix_Body[idx, :] = embedding_vector 
        
w2v_matrix_Body.shape         

(1000, 100)

In [71]:
X1_feat = np.dot(X1_arr ,  w2v_matrix_title)  
X2_feat = np.dot(X2_arr ,  w2v_matrix_Body )       
# X_feat = hstack([X1_feat,X2_feat])     
X_feat_w2v = np.c_[X1_feat , X2_feat] 
print(X_feat_w2v.shape)                             

(63167, 200)


In [72]:
X_train, X_test, y_train, y_test = train_test_split(X_feat_w2v, y_bin, test_size = 0.2, random_state = 0)   
X_train.shape , X_test.shape   , y_train.shape        

((50533, 200), (12634, 200), (50533, 100))

In [68]:
print("Results for tf-idf weighted trained w2v features 100dim") 
print('------------------------------')  

dummy = DummyClassifier()
sgd = SGDClassifier()
lr = LogisticRegression()
# svc = LinearSVC()

for classifier in [dummy, sgd, lr]:   
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)  
    print_score(y_pred, classifier)                                

Results for tf-idf weighted trained w2v features 100dim
------------------------------
Clf:  DummyClassifier
Jacard score: 0.0
Hamming loss: 1.5717112553427262
---
Clf:  SGDClassifier
Jacard score: 33.64670004171024
Hamming loss: 1.5836631312331804
---
Clf:  LogisticRegression
Jacard score: 36.46377499868081
Hamming loss: 1.1972455279404781
---


We can see by compairing performance for pre-trained W2V features and when trained on text data. We got better results when using features obtained by training on text corpus  